In [ ]:
#load libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import copy
import seaborn as sns
import importlib
from matplotlib import cm

In [ ]:
#load cmif libraries
os.chdir('/mnt/c/Users/engje/Documents/Data/')
from cmif.mplex_image import visualize as viz, process, preprocess, gating

## Notes

add notes about choices for gating strategy

## Load Data

In [ ]:
#set location of files
rootdir = '/mnt/c/Users/engje/Documents/Data/cmIF_2021-07-07_RS-mTMA'
# go to location of files
os.chdir(rootdir)
preprocess.cmif_mkdir(['GatingPlots'])
#os.listdir()

In [ ]:
# 3 define samples to work with/ image combos
ls_sample = ['20211109_RS-mTMA-5']

In [ ]:
df_data = pd.DataFrame()
for s_sample in ls_sample:
    df_data = df_data.append(pd.read_csv(f'{s_sample}_ManualPositive.csv',index_col=0))

## Specify Gating Strategy

In [ ]:
df_data.columns

In [ ]:
#parameters

# cell types
ls_endothelial = ['CD31']
ls_immune = ['CD45'] 
ls_tumor = ['Ecad','EpCAM','CK5'] 
ls_prolif = ['Ki67']

#tcell/myeloid
s_tcell = 'CD8' 
s_bcell = 'CD11c' #dendritic cells
s_myeloid = 'CD11b'
ls_immune_functional = ['FoxP3','CD103'] # not in dataset: 'FoxP3_Nuclei','GRNZB_Nuclei',

#luminal/basal/mesenchymal
ls_luminal = ['SOX9'] # not in dataset 'CK8_Ring'
ls_basal = ['CK5'] 
ls_mes = [] 
ls_tumor_plus = [] #['CD44_Ring'] + ['Lum','Bas','Mes']
ls_stromal_function =  ['Vim','aSMA','ColIV', 'ColVI','Gal3']
ls_tumor_prolif = ['pMYC','Ki67']  

#index of cell line samples (i.e. 100% tumor)
ls_cellline_index = []


In [ ]:
from mplex_image import analyze

In [ ]:
def main_celltypes(df_data,ls_endothelial,ls_immune,ls_tumor,ls_cellline_index):
    #celltpye
    #1 endothelial
    df_data['endothelial'] = df_data.loc[:,ls_endothelial].any(axis=1)
    #3 tumor
    ls_exclude =  ls_endothelial 
    df_data['tumor'] = df_data.loc[:,ls_tumor].any(axis=1) & ~df_data.loc[:,ls_exclude].any(axis=1) 
    #2 immune
    ls_exclude = ls_endothelial + ls_tumor
    df_data['immune'] = df_data.loc[:,ls_immune].any(axis=1) & ~df_data.loc[:,ls_exclude].any(axis=1)
    #4 stromal
    ls_exclude = ls_immune + ls_endothelial + ls_tumor
    df_data['stromal'] = ~df_data.loc[:,ls_exclude].any(axis=1)
    #add celltype
    ls_cell_names = ['stromal','endothelial','tumor','immune']
    s_type_name = 'celltype'
    analyze.add_celltype(df_data, ls_cell_names, s_type_name)
    #fix cell lines (all tumor!)
    df_data['slide_scene'] = [item.split('_cell')[0] for item in df_data.index]
    df_data.loc[df_data[df_data.slide_scene.isin(ls_cellline_index)].index,'celltype'] = 'tumor'
    df_data['immune'] = df_data.loc[:,'celltype'] == 'immune'
    df_data['stromal'] = df_data.loc[:,'celltype'] == 'stromal'
    df_data['endothelial'] = df_data.loc[:,'celltype'] == 'endothelial'
    return(df_data)

def immune_types(df_data,s_myeloid,s_bcell,s_tcell):
    ## T cell, B cell or myeloid
    df_data['Myeloid'] = df_data.loc[:,[s_myeloid,'immune']].all(axis=1) 
    df_data['DendriticCell'] = df_data.loc[:,[s_bcell,'immune']].all(axis=1) & ~df_data.loc[:,['Myeloid',s_tcell]].any(axis=1)
    df_data['CD8Tcell'] = df_data.loc[:,[s_tcell,'immune']].all(axis=1) & ~df_data.loc[:,['DendriticCell','Myeloid']].any(axis=1)
    df_data['OtherImmune'] = df_data.loc[:,'immune'] & ~df_data.loc[:,['DendriticCell','CD8Tcell','Myeloid']].any(axis=1)
    ## CD4 and CD8 
    if df_data.columns.isin(['CD8_Ring','CD4_Ring']).sum()==2:
        print('CD4 AND CD8')
        df_data['CD8Tcell'] = df_data.loc[: ,['CD8_Ring','TcellImmune']].all(axis=1)
        df_data['CD4Tcell'] = df_data.loc[: ,['CD4_Ring','TcellImmune']].all(axis=1) & ~df_data.loc[: ,'CD8Tcell']
        df_data['UnspecifiedTcell'] = df_data.TcellImmune & ~df_data.loc[:,['CD8Tcell','CD4Tcell']].any(axis=1) #if cd4 or 8 then sum = 2
        ## check
        ls_immune = df_data[df_data.loc[:,'TcellImmune']].index.tolist()
        if ((df_data.loc[ls_immune,['CD8Tcell','CD4Tcell','UnspecifiedTcell']].sum(axis=1)!=1)).any():
            print('Error in Tcell cell types')
        ls_immuntype = ['CD68Mac','CD20Bcell','UnspecifiedImmune','CD8Tcell','CD4Tcell','UnspecifiedTcell'] #'TcellImmune',
    else:
        ls_immuntype = ['Myeloid','DendriticCell','OtherImmune','CD8Tcell'] #'TcellImmune',
    #add Immunetype
    ls_cell_names = ls_immuntype
    s_type_name = 'ImmuneType'
    analyze.add_celltype(df_data, ls_cell_names, s_type_name)

    #get rid of unspecfied immune cells (make them stroma)
    #ls_index = df_data[df_data.ImmuneType.fillna('x').str.contains('Unspecified')].index
    #df_data.loc[ls_index,'celltype'] = 'stromal'
    #df_data.loc[ls_index,'ImmuneType'] = np.nan
    #df_data.loc[ls_index,'stromal'] = True
    #df_data.loc[ls_index,'immune'] = False
    return(df_data)


## Perform Gating

In [ ]:
%%capture
#custom gating
df_data = main_celltypes(df_data,ls_endothelial,ls_immune,ls_tumor,ls_cellline_index)
df_data = gating.proliferation(df_data,ls_prolif)
df_data = immune_types(df_data,s_myeloid,s_bcell,s_tcell)
df_data = gating.immune_functional(df_data,ls_immune_functional)
df_data = gating.cell_prolif(df_data)
#df_data = gating.diff_hr_state(df_data,ls_luminal,ls_basal,ls_mes)

In [ ]:
%%capture
#general gating
df_data = gating.celltype_gates(df_data,ls_tumor_prolif,s_new_name='TumorProlif',s_celltype='tumor')
df_data = gating.celltype_gates(df_data,ls_tumor_plus,s_new_name='TumorDiffPlus',s_celltype='tumor')
df_data = gating.celltype_gates(df_data,ls_stromal_function,s_new_name='StromalType',s_celltype='stromal')

In [ ]:
df_data = gating.non_tumor(df_data)

## Output Gating Data

In [ ]:
#check
df_data.loc[:,df_data.dtypes=='object']

In [ ]:
#drop extra colums
df_gate = df_data.loc[:,df_data.dtypes!='bool']

In [ ]:
ls_clow = ['RS-mTMA-5_sceneI11','RS-mTMA-5_sceneG09','RS-mTMA-5_sceneH11']
for s_scene in ls_clow:
    ls_index = df_gate[(df_gate.slide_scene==s_scene) & (df_gate.StromalType == 'stromal___')].index
    df_gate.loc[ls_index,'celltype'] = 'tumor'
    df_gate.loc[ls_index,'NonTumor'] = np.NaN
    df_gate.loc[ls_index,'StromalType'] = np.NaN
    df_gate.loc[ls_index,'NonTumorFunc'] = np.NaN
    df_gate.loc[ls_index,'CellProlif'] = df_gate.loc[ls_index,'celltype'] + '_' + df_gate.loc[ls_index,'proliferation']
    df_gate.loc[ls_index,'TumorDiffPlus'] = 'Claudin_Low'
    #break

In [ ]:
s_scene

In [ ]:
s_out = '20211111_RS-mTMA-5'
if not os.path.exists(f'{s_out}_GatedPositiveCellNames.csv'):
    print('saving new csv')
    df_gate.to_csv(f'{s_out}_GatedPositiveCellNames.csv')

## Plot


In [ ]:
#importlib.reload(viz)

In [ ]:
df_data = pd.read_csv(f'{s_out}_GatedPositiveCellNames.csv',index_col=0)

In [ ]:
df_data['scene'] = [item.split('_cell')[0] for item in df_data.index]
#make a fake scene
df_data.loc[df_data[df_data.DAPI_Y > df_data.DAPI_Y.median()].index,'scene'] = 'BC44290-146_scene2'

In [ ]:
%matplotlib inline
s_grouper='scene'

#calculate proportions
for s_cell in df_data.columns[(df_data.dtypes=='object') & ~(df_data.columns.isin([s_grouper]))].tolist():
    df_prop = viz.prop_positive(df_data,s_cell=s_cell,s_grouper=s_grouper)
    df_prop.rename(index=dict(zip(df_prop.index,['top','bottom'])),inplace=True) 

    # make annotations
    df_annot=pd.DataFrame(data={'ID': df_prop.index.tolist()},index=df_prop.index)
    lut = dict(zip(sorted(df_annot.ID.unique()),cm.tab10.colors))
    g, df_plot_less = viz.prop_clustermap(df_prop,df_annot,i_thresh =.01,lut=lut)
    g.savefig(f'./GatingPlots/{s_cell}_clustermap.png',dpi=150)
    plt.close()
    fig = viz.prop_barplot(df_plot_less,s_cell,colormap="Spectral")
    fig.savefig(f'./GatingPlots/{s_cell}_bar.png',dpi=200)